In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.base import TransformerMixin
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,roc_auc_score,classification_report,roc_curve,auc, f1_score

import import_ipynb
from function_for_eda import *

C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


importing Jupyter notebook from function_for_eda.ipynb


# 6. CREDIT_CARD_BALANCE.CSV
Bảng credit_card_blance chứa các thông tin về những credit card trước đây mà khách hàng có tại Home Credit. Từ các bước EDA trước đó, chúng ta tiến hành các bước feature engineering sau:
- <b>Bước 1</b>: Xóa bỏ các giá trị bất thường
- <b>Bước 2</b>: Tạo các feature mới như total drawings, number of drawings, blance to limit ratio, payment done to minimum, payment required difference
- <b>Bước 3</b>: Thực hiện các aggregations thông qua SK_ID_PREV bằng các phép aggregation như min, max, sum, count,....Đầu tiên nhóm 12 aggregate trên toàn bộ data, sau đó aggregate những bản ghi trong 2 năm gần nhất và và aggregate qua những năm còn lại. Cuối cùng là aggregate data qua column Name_contract_Type. Việc lựa chọn các phép aggregation dựa trên domain knowledge, quá trình eda, cũng như một số solutions tham khảo khác
- <b>Bước 4</b>: Thực hiện các aggregations thông qua SK_ID_CURR bằng các phép aggregation cho toàn bộ data. Do quá trình EDA chúng ta phát hiện hầu như khách hàng chỉ có 1 credit card trước đó, nên ở đây chúng ta chỉ cần dùng phép aggregation mean.

In [2]:
cc_balance = pd.read_csv('../dseb63_final_project_DP_dataset/dseb63_credit_card_balance.csv')
cc_balance

,SK_ID_PREV,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,...,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,SK_ID_CURR
0,2582071,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.0,2250.0,...,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0,87788
1,2582071,-82,16809.210,67500,0.0,0.0,0.0,0.0,3375.0,9000.0,...,16809.210,0.0,0,0.0,0.0,18.0,Active,0,0,87788
2,2582071,-84,27577.890,67500,0.0,0.0,0.0,0.0,3375.0,4500.0,...,27577.890,0.0,0,0.0,0.0,16.0,Active,0,0,87788
3,2582071,-7,65159.235,45000,0.0,0.0,0.0,0.0,2250.0,2250.0,...,65609.235,0.0,0,0.0,0.0,63.0,Active,0,0,87788
4,2582071,-59,70475.850,67500,24750.0,24750.0,0.0,0.0,3375.0,4500.0,...,70475.850,4.0,4,0.0,0.0,41.0,Active,0,0,87788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3227960,1551072,-1,0.000,202500,NaN,0.0,NaN,NaN,0.0,NaN,...,0.000,NaN,0,NaN,NaN,0.0,Active,0,0,259579
3227961,2339982,-1,0.000,45000,0.0,0.0,0.0,0.0,0.0,NaN,...,0.000,0.0,0,0.0,0.0,0.0,Active,0,0,220697
3227962,2720102,-1,0.000,225000,NaN,0.0,NaN,NaN,0.0,NaN,...,0.000,NaN,0,NaN,NaN,0.0,Active,0,0,93573
3227963,1897864,-2,0.000,270000,NaN,0.0,NaN,NaN,0.0,NaN,...,0.000,NaN,0,NaN,NaN,0.0,Active,0,0,11072


In [3]:
#there is one abruptly large value for AMT_PAYMENT_CURRENT
cc_balance['AMT_PAYMENT_CURRENT'][cc_balance['AMT_PAYMENT_CURRENT'] > 4000000] = np.nan
#calculating the total missing values for each previous credit card
cc_balance['MISSING_VALS_TOTAL_CC'] = cc_balance.isna().sum(axis = 1)
#making the MONTHS_BALANCE Positive
cc_balance['MONTHS_BALANCE'] = np.abs(cc_balance['MONTHS_BALANCE'])
#sorting the DataFrame according to the month of status from oldest to latest, for rolling computations
cc_balance = cc_balance.sort_values(by = ['SK_ID_PREV','MONTHS_BALANCE'], ascending = [1,0])

#Creating new features
cc_balance['AMT_DRAWING_SUM'] = cc_balance['AMT_DRAWINGS_ATM_CURRENT'] + cc_balance['AMT_DRAWINGS_CURRENT'] + cc_balance[
    'AMT_DRAWINGS_OTHER_CURRENT'] + cc_balance['AMT_DRAWINGS_POS_CURRENT']
cc_balance['BALANCE_LIMIT_RATIO'] = cc_balance['AMT_BALANCE'] / (cc_balance['AMT_CREDIT_LIMIT_ACTUAL'] + 0.00001)
cc_balance['CNT_DRAWING_SUM'] = cc_balance['CNT_DRAWINGS_ATM_CURRENT'] + cc_balance['CNT_DRAWINGS_CURRENT'] + cc_balance[
    'CNT_DRAWINGS_OTHER_CURRENT'] + cc_balance['CNT_DRAWINGS_POS_CURRENT'] + cc_balance['CNT_INSTALMENT_MATURE_CUM']
cc_balance['MIN_PAYMENT_RATIO'] = cc_balance['AMT_PAYMENT_CURRENT'] / (cc_balance['AMT_INST_MIN_REGULARITY'] + 0.0001)
cc_balance['PAYMENT_MIN_DIFF'] = cc_balance['AMT_PAYMENT_CURRENT'] - cc_balance['AMT_INST_MIN_REGULARITY']
cc_balance['MIN_PAYMENT_TOTAL_RATIO'] = cc_balance['AMT_PAYMENT_TOTAL_CURRENT'] / (cc_balance['AMT_INST_MIN_REGULARITY'] +0.00001)
cc_balance['PAYMENT_MIN_DIFF'] = cc_balance['AMT_PAYMENT_TOTAL_CURRENT'] - cc_balance['AMT_INST_MIN_REGULARITY']
cc_balance['AMT_INTEREST_RECEIVABLE'] = cc_balance['AMT_TOTAL_RECEIVABLE'] - cc_balance['AMT_RECEIVABLE_PRINCIPAL']
cc_balance['SK_DPD_RATIO'] = cc_balance['SK_DPD'] / (cc_balance['SK_DPD_DEF'] + 0.00001)

        #calculating the rolling Exponential Weighted Moving Average over months for certain features
    

In [4]:
rolling_columns = [
        'AMT_BALANCE',
        'AMT_CREDIT_LIMIT_ACTUAL',
        'AMT_RECEIVABLE_PRINCIPAL',
        'AMT_RECIVABLE',
        'AMT_TOTAL_RECEIVABLE',
        'AMT_DRAWING_SUM',
        'BALANCE_LIMIT_RATIO',
        'CNT_DRAWING_SUM',
        'MIN_PAYMENT_RATIO',
        'PAYMENT_MIN_DIFF',
        'MIN_PAYMENT_TOTAL_RATIO',
        'AMT_INTEREST_RECEIVABLE',
        'SK_DPD_RATIO' ]
exp_weighted_columns = ['EXP_' + ele for ele in rolling_columns]
cc_balance[exp_weighted_columns] = cc_balance.groupby(['SK_ID_CURR','SK_ID_PREV'])[rolling_columns].transform(lambda x: x.ewm(alpha = 0.7).mean())




In [5]:
overall_aggregations = {
            'SK_ID_CURR' : ['first'],
            'MONTHS_BALANCE': ['max'],
            'AMT_BALANCE' : ['sum','mean','max'],
            'AMT_CREDIT_LIMIT_ACTUAL' : ['sum','mean','max'],
            'AMT_DRAWINGS_ATM_CURRENT' : ['sum','max'],
            'AMT_DRAWINGS_CURRENT' : ['sum','max'],
            'AMT_DRAWINGS_OTHER_CURRENT' : ['sum','max'],
            'AMT_DRAWINGS_POS_CURRENT' : ['sum','max'],
            'AMT_INST_MIN_REGULARITY' : ['mean','min','max'],
            'AMT_PAYMENT_CURRENT' : ['mean','min','max'],
            'AMT_PAYMENT_TOTAL_CURRENT' : ['mean','min','max'],
            'AMT_RECEIVABLE_PRINCIPAL' : ['sum','mean','max'],
            'AMT_RECIVABLE' : ['sum','mean','max'],
            'AMT_TOTAL_RECEIVABLE' : ['sum','mean','max'],
            'CNT_DRAWINGS_ATM_CURRENT' : ['sum','max'],
            'CNT_DRAWINGS_CURRENT' : ['sum','max'],
            'CNT_DRAWINGS_OTHER_CURRENT' : ['sum','max'],
            'CNT_DRAWINGS_POS_CURRENT' : ['sum','max'],
            'CNT_INSTALMENT_MATURE_CUM' : ['sum','max','min'],
            'SK_DPD' : ['sum','max'],
            'SK_DPD_DEF' : ['sum','max'],

            'AMT_DRAWING_SUM' : ['sum','max'],
            'BALANCE_LIMIT_RATIO' : ['mean','max','min'],
            'CNT_DRAWING_SUM' : ['sum','max'],
            'MIN_PAYMENT_RATIO': ['min','mean'],
            'PAYMENT_MIN_DIFF' : ['min','mean'],
            'MIN_PAYMENT_TOTAL_RATIO' : ['min','mean'], 
            'AMT_INTEREST_RECEIVABLE' : ['min','mean'],
            'SK_DPD_RATIO' : ['max','mean'],

            'EXP_AMT_BALANCE' : ['last'],
            'EXP_AMT_CREDIT_LIMIT_ACTUAL' : ['last'],
            'EXP_AMT_RECEIVABLE_PRINCIPAL' : ['last'],
            'EXP_AMT_RECIVABLE' : ['last'],
            'EXP_AMT_TOTAL_RECEIVABLE' : ['last'],
            'EXP_AMT_DRAWING_SUM' : ['last'],
            'EXP_BALANCE_LIMIT_RATIO' : ['last'],
            'EXP_CNT_DRAWING_SUM' : ['last'],
            'EXP_MIN_PAYMENT_RATIO' : ['last'],
            'EXP_PAYMENT_MIN_DIFF' : ['last'],
            'EXP_MIN_PAYMENT_TOTAL_RATIO' : ['last'],
            'EXP_AMT_INTEREST_RECEIVABLE' : ['last'],
            'EXP_SK_DPD_RATIO' : ['last'],
            'MISSING_VALS_TOTAL_CC' : ['sum']
        }
aggregations_for_categories = {
    'SK_DPD' : ['sum','max'],
    'SK_DPD_DEF' : ['sum','max'],
    'BALANCE_LIMIT_RATIO' : ['mean','max','min'],
    'CNT_DRAWING_SUM' : ['sum','max'],
    'MIN_PAYMENT_RATIO': ['min','mean'],
    'PAYMENT_MIN_DIFF' : ['min','mean'],
    'MIN_PAYMENT_TOTAL_RATIO' : ['min','mean'], 
    'AMT_INTEREST_RECEIVABLE' : ['min','mean'],
    'SK_DPD_RATIO' : ['max','mean'],
    'EXP_AMT_DRAWING_SUM' : ['last'],
    'EXP_BALANCE_LIMIT_RATIO' : ['last'],
    'EXP_CNT_DRAWING_SUM' : ['last'],
    'EXP_MIN_PAYMENT_RATIO' : ['last'],
    'EXP_PAYMENT_MIN_DIFF' : ['last'],
    'EXP_MIN_PAYMENT_TOTAL_RATIO' : ['last'],
    'EXP_AMT_INTEREST_RECEIVABLE' : ['last'],
    'EXP_SK_DPD_RATIO' : ['last']
}
aggregations_for_year = {
    'SK_DPD' : ['sum','max'],
    'SK_DPD_DEF' : ['sum','max'],
    'BALANCE_LIMIT_RATIO' : ['mean','max','min'],
    'CNT_DRAWING_SUM' : ['sum','max'],
    'MIN_PAYMENT_RATIO': ['min','mean'],
    'PAYMENT_MIN_DIFF' : ['min','mean'],
    'MIN_PAYMENT_TOTAL_RATIO' : ['min','mean'], 
    'AMT_INTEREST_RECEIVABLE' : ['min','mean'],
    'SK_DPD_RATIO' : ['max','mean'],
    'EXP_AMT_DRAWING_SUM' : ['last'],
    'EXP_BALANCE_LIMIT_RATIO' : ['last'],
    'EXP_CNT_DRAWING_SUM' : ['last'],
    'EXP_MIN_PAYMENT_RATIO' : ['last'],
    'EXP_PAYMENT_MIN_DIFF' : ['last'],
    'EXP_MIN_PAYMENT_TOTAL_RATIO' : ['last'],
    'EXP_AMT_INTEREST_RECEIVABLE' : ['last'],
    'EXP_SK_DPD_RATIO' : ['last']
}

In [6]:
#performing overall aggregations over SK_ID_PREV for all features
cc_balance_aggregated_overall = cc_balance.groupby('SK_ID_PREV').agg(overall_aggregations)
cc_balance_aggregated_overall.columns = ['_'.join(ele).upper() for ele in cc_balance_aggregated_overall.columns]
cc_balance_aggregated_overall.rename(columns = {'SK_ID_CURR_FIRST' : 'SK_ID_CURR'}, inplace = True)

#aggregating over SK_ID_PREV for different categories
contract_status_categories = ['Active','Completed']
cc_balance_aggregated_categories = pd.DataFrame()
for i, contract_type in enumerate(contract_status_categories):
    group = cc_balance[cc_balance['NAME_CONTRACT_STATUS'] == contract_type].groupby('SK_ID_PREV').agg(aggregations_for_categories)
    group.columns = ['_'.join(ele).upper() + '_' + contract_type.upper() for ele in group.columns]
    if i == 0:
        cc_balance_aggregated_categories = group
    else:
        cc_balance_aggregated_categories = cc_balance_aggregated_categories.merge(group, on = 'SK_ID_PREV', how = 'outer')
#aggregating over SK_ID_PREV for rest of the categories
cc_balance_aggregated_categories_rest = cc_balance[(cc_balance['NAME_CONTRACT_STATUS'] != 'Active') & 
                                                        (cc_balance.NAME_CONTRACT_STATUS != 'Completed')].groupby('SK_ID_PREV').agg(aggregations_for_categories)
cc_balance_aggregated_categories_rest.columns = ['_'.join(ele).upper() + '_REST' for ele in cc_balance_aggregated_categories_rest.columns]
#merging all the categorical aggregations
cc_balance_aggregated_categories = cc_balance_aggregated_categories.merge(cc_balance_aggregated_categories_rest, on = 'SK_ID_PREV', how = 'outer')

#aggregating over SK_ID_PREV for different years
cc_balance['YEAR_BALANCE'] = cc_balance['MONTHS_BALANCE'] //12
cc_balance_aggregated_year = pd.DataFrame()
for year in range(2):
    group = cc_balance[cc_balance['YEAR_BALANCE'] == year].groupby('SK_ID_PREV').agg(aggregations_for_year)
    group.columns = ['_'.join(ele).upper() + '_YEAR_' + str(year) for ele in group.columns]
    if year == 0:
        cc_balance_aggregated_year = group
    else:
        cc_balance_aggregated_year = cc_balance_aggregated_year.merge(group, on = 'SK_ID_PREV', how = 'outer')
        
#aggregating over SK_ID_PREV for rest of years
cc_balance_aggregated_year_rest = cc_balance[cc_balance['YEAR_BALANCE'] >= 2].groupby('SK_ID_PREV').agg(aggregations_for_year)
cc_balance_aggregated_year_rest.columns = ['_'.join(ele).upper() + '_YEAR_REST' for ele in cc_balance_aggregated_year_rest.columns]

#merging all the yearwise aggregations
cc_balance_aggregated_year = cc_balance_aggregated_year.merge(cc_balance_aggregated_year_rest, on = 'SK_ID_PREV', how = 'outer')
cc_balance = cc_balance.drop('YEAR_BALANCE', axis = 1)

#merging all the aggregations
cc_aggregated = cc_balance_aggregated_overall.merge(cc_balance_aggregated_categories, on = 'SK_ID_PREV', how = 'outer')
cc_aggregated = cc_aggregated.merge(cc_balance_aggregated_year, on = 'SK_ID_PREV', how = 'outer')

#one-hot encoding the categorical column NAME_CONTRACT_STATUS
name_contract_dummies = pd.get_dummies(cc_balance.NAME_CONTRACT_STATUS, prefix='CONTRACT')
contract_names = name_contract_dummies.columns.tolist()   

#merging the one-hot encoded feature with original table
cc_balance = pd.concat([cc_balance, name_contract_dummies], axis=1)
#aggregating over SK_ID_PREV the one-hot encoded columns
aggregated_cc_contract = cc_balance[['SK_ID_PREV'] + contract_names].groupby('SK_ID_PREV').mean()

#merging with the aggregated table
cc_aggregated = cc_aggregated.merge(aggregated_cc_contract, on = 'SK_ID_PREV', how = 'outer')

#now we will aggregate on SK_ID_CURR
#As seen from EDA, since most of the SK_ID_CURR had only 1 credit card, so for aggregations, we will simply take the means
cc_aggregated = cc_aggregated.groupby('SK_ID_CURR', as_index = False).mean()


In [7]:
cc_aggregated

,SK_ID_CURR,MONTHS_BALANCE_MAX,AMT_BALANCE_SUM,AMT_BALANCE_MEAN,AMT_BALANCE_MAX,AMT_CREDIT_LIMIT_ACTUAL_SUM,AMT_CREDIT_LIMIT_ACTUAL_MEAN,AMT_CREDIT_LIMIT_ACTUAL_MAX,AMT_DRAWINGS_ATM_CURRENT_SUM,AMT_DRAWINGS_ATM_CURRENT_MAX,...,EXP_MIN_PAYMENT_TOTAL_RATIO_LAST_YEAR_REST,EXP_AMT_INTEREST_RECEIVABLE_LAST_YEAR_REST,EXP_SK_DPD_RATIO_LAST_YEAR_REST,CONTRACT_Active,CONTRACT_Approved,CONTRACT_Completed,CONTRACT_Demand,CONTRACT_Refused,CONTRACT_Sent proposal,CONTRACT_Signed
0,0,8.0,53884.035,6735.504375,36569.430,1440000.0,180000.000000,180000.0,0.0,0.0,...,NaN,NaN,NaN,1.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
1,1,9.0,1324627.920,147180.880000,227526.210,2025000.0,225000.000000,225000.0,234000.0,180000.0,...,NaN,NaN,NaN,1.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
2,3,11.0,1368684.630,124425.875455,140704.965,1485000.0,135000.000000,135000.0,0.0,0.0,...,NaN,NaN,NaN,0.909091,0.0,0.000000,0.0,0.0,0.0,0.090909
3,7,18.0,0.000,0.000000,0.000,1125000.0,62500.000000,225000.0,0.0,NaN,...,NaN,NaN,NaN,0.833333,0.0,0.166667,0.0,0.0,0.0,0.000000
4,9,11.0,4711.500,428.318182,4711.500,1800000.0,163636.363636,900000.0,68400.0,68400.0,...,NaN,NaN,NaN,0.181818,0.0,0.818182,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86900,307501,9.0,0.000,0.000000,0.000,1732500.0,247500.000000,247500.0,0.0,NaN,...,NaN,NaN,NaN,1.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
86901,307504,96.0,831790.260,8664.481875,50528.790,6187500.0,64453.125000,67500.0,0.0,0.0,...,1.922900e-19,-5.688251e-06,1.106339e-21,1.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
86902,307505,9.0,0.000,0.000000,0.000,810000.0,135000.000000,135000.0,0.0,NaN,...,NaN,NaN,NaN,1.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
86903,307508,18.0,0.000,0.000000,0.000,16200000.0,900000.000000,900000.0,0.0,NaN,...,NaN,NaN,NaN,0.944444,0.0,0.000000,0.0,0.0,0.0,0.055556


In [8]:
cc_aggregated.to_csv('credit_cat_final.csv', index = False)
print('done')

done
